# Equations of motion for the cat-bot

Do imports.

In [103]:
import sympy as sym
import numpy as np
from IPython.display import display, Markdown
from sympy.physics import mechanics
mechanics.init_vprinting()

Define variables and constants as symbols.

In [121]:
# Time
t = sym.Symbol('t')

# Horizontal position of wheel axle
zeta = mechanics.dynamicsymbols('zeta', real=True)

# Angle of body from vertical (positive means leaning forward)
theta = mechanics.dynamicsymbols('theta', real=True)

# Torque applied by the body on the wheel
tau = sym.symbols('tau', real=True)

# Wheel parameters
#   m_w  mass
#   J_w  moment of inertia
#   r_w  radius
m_w, J_w, r_w = sym.symbols('m_w, J_w, r_w', real=True, positive=True)

# Body parameters
#   m_b  mass
#   J_b  moment of inertia (about center-of-mass)
#   r_b  distance between wheel axle and body center-of-mass
m_b, J_b, r_b = sym.symbols('m_b, J_b, r_b', real=True, positive=True)

# Acceleration of gravity
g = sym.symbols('g', real=True, positive=True)

Compute Lagrangian.

In [122]:
# Position of wheel and body
p_w = sym.Matrix([zeta, r_w])
p_b = sym.Matrix([zeta + r_b * sym.sin(theta), r_w + r_b * sym.cos(theta)])

# Linear velocity of wheel and body
v_w = p_w.diff(t)
v_b = p_b.diff(t)

# Angular velocity of wheel (assume rolling without slipping on flat ground) and body
omega_w = zeta.diff(t) / r_w
omega_b = theta.diff(t)

# Kinetic and potential energy
T = (m_w * v_w.dot(v_w) + m_b * v_b.dot(v_b) + J_w * omega_w**2 + J_b * omega_b**2) / 2
V = (m_w * p_w[1] * g) + (m_b * p_b[1] * g)

# Lagrangian
L = sym.Matrix([sym.simplify(T - V)])

Compute and show the matrices $M(q)$, $N(q, \dot{q})$, and $F(q)$ for which the equations of motion can be expressed as

$$ M(q) \ddot{q} + N(q, \dot{q}) = F(q) r $$

where

$$ q = \begin{bmatrix} \zeta \\ \theta \end{bmatrix} \qquad\text{and}\qquad r = \begin{bmatrix} \tau \end{bmatrix}. $$

In [123]:
# Generalized coordinates
q = sym.Matrix([zeta, theta])

# Generalized velocities
v = q.diff(t)

# Coefficients in the equations of motion
M = sym.simplify(L.jacobian(v).jacobian(v))
N = sym.simplify(L.jacobian(v).jacobian(q) @ v - L.jacobian(q).T)
F = sym.simplify(sym.Matrix([(zeta / r_w) - theta]).jacobian(q).T)

# Show results
display(Markdown(f'$$ M(q) = {mechanics.mlatex(M)} $$'))
display(Markdown(f'$$ N(q, \\dot{{q}}) = {mechanics.mlatex(N)} $$'))
display(Markdown(f'$$ F(q) = {mechanics.mlatex(F)} $$'))

$$ M(q) = \left[\begin{matrix}\frac{J_{w}}{r_{w}^{2}} + m_{b} + m_{w} & m_{b} r_{b} \cos{\left(\theta \right)}\\m_{b} r_{b} \cos{\left(\theta \right)} & J_{b} + m_{b} r_{b}^{2}\end{matrix}\right] $$

$$ N(q, \dot{q}) = \left[\begin{matrix}- m_{b} r_{b} \sin{\left(\theta \right)} \dot{\theta}^{2}\\- g m_{b} r_{b} \sin{\left(\theta \right)}\end{matrix}\right] $$

$$ F(q) = \left[\begin{matrix}\frac{1}{r_{w}}\\-1\end{matrix}\right] $$

In [129]:
# m matrix
m = sym.Matrix([q,v])
m_dot = m.diff(t)

# Torque Matrix
n = sym.Matrix([tau])

# m_dot = f(m,n) vector
f = sym.Matrix([ v , M.inv() @ F @ n - M.inv() @ N ])

# finding the A and B matrices for linearization: x_dot = A@x + B@u
A_sym = sym.simplify(f.jacobian([m]))
B_sym = sym.simplify(f.jacobian([n]))
A_num = sym.lambdify((zeta,theta,zeta.diff(t),theta.diff(t),tau,r_w, m_w, J_w, r_b, m_b, J_b, g), A_sym)
B_num = sym.lambdify((zeta,theta,zeta.diff(t),theta.diff(t),tau,r_w, m_w, J_w, r_b, m_b, J_b, g), B_sym)
A = A_num(0.,0.,0.,0.,0.,.325,2.4,.12675,0.3,12.,.8,9.81)
B = B_num(0.,0.,0.,0.,0.,.325,2.4,.12675,0.3,12.,.8,9.81)

# Solving for R or [tau]
# Rearange m_dot = A@m + B@R --> B@R = m_dot - A@m
# R = A_sym @ m - m_dot
# R[2] = R[2] / B_sym[2]
# R[3] = R[3] / B_sym[3]
# R = sym.simplify(R)



In [130]:
A,B

(array([[ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ],
        [ 0.        , -7.76744868,  0.        ,  0.        ],
        [ 0.        , 33.65894428,  0.        , -0.        ]]),
 array([[ 0.        ],
        [ 0.        ],
        [ 0.57335138],
        [-1.62982179]]))

Define and show the numerical value of constants (consistant with the URDF file).

In [109]:
##########################################
# Temporary variables that can be ignored

# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

##########################################
# Parameters

# Define them
params = {
    r_w: r,
    m_w: 2 * mw,
    J_w: 2 * Jw,
    r_b: h,
    m_b: mb,
    J_b: Jby,
    g: 9.81,
}

# Show them
s = ''
for key, val in params.items():
    s += fr'{key} &= {mechanics.mlatex(val)} \\ '
s = s[:-3]
display(Markdown(fr'$$ \begin{{align*}}{s}\end{{align*}} $$'))

$$ \begin{align*}r_w &= 0.325 \\ m_w &= 2.4 \\ J_w &= 0.12675 \\ r_b &= 0.3 \\ m_b &= 12.0 \\ J_b &= 0.8 \\ g &= 9.81 \end{align*} $$

In [ ]:
# f_num = sym.lambdify([q, v, tau], f)
# f_num(q_e, v_e, tau_e)

# M_num = sym.lambdify([theta,r_w, m_w, J_w, r_b, m_b, J_b, g], M)
# N_num = sym.lambdify([zeta, theta, zeta.diff(t), theta.diff(t), r_w, m_w, J_w, r_b, m_b, J_b, g], N)
# F_num = sym.lambdify([r_w], F)
# zeta_e, theta_e = sym.symbols('zeta_e, theta_e')
# M_num(1,1,1,1,1,1,11,1,1,1,1) , N_num(1,1,1,1,1,1,1,1,1,1,1) , F_num(1,1,1,1,1,1,1,1,1,1,1)

In [116]:
rw = r
mw = 2 * mw
Jw = 2 * Jw
rb = h
mb = mb
Jb = Jby
g  = 9.81

M = sym.Matrix([ [ Jw / (rw**2) + mb + mw , mb*rb*sym.cos(theta) ] ,
                 [ mb*rb*sym.cos(theta)   , Jb + mb*(rb**2)      ] ])
N = sym.Matrix([ [ -mb*rb*sym.sin(theta)*(theta.diff(t)**2) ] ,
                 [ -g*mb*rb*sym.sin(theta)                  ] ])
F = sym.Matrix([ 1/rw , -1 ])

In [120]:
# m matrix
m = sym.Matrix([q,v])
m_dot = m.diff(t)

# Torque Matrix
n = sym.Matrix([tau])

# m_dot = f(m,n) vector
f = sym.simplify(sym.Matrix([ v , M.inv() @ F @ n - M.inv() @ N ]))

# finding the A and B matrices for linearization: x_dot = A@x + B@u
A_sym = sym.simplify(f.jacobian([m]))
B_sym = sym.simplify(f.jacobian([n]))

A_sym , B_sym

⎛⎡0                                                                            ↪
⎜⎢                                                                             ↪
⎜⎢0                                                                            ↪
⎜⎢                                                                             ↪
⎜⎢                           ⎛                      ⎛         2            ⎞ ⎛ ↪
⎜⎢          1.19306717673448⋅⎝- 0.00440553821187624⋅⎝12.96⋅cos (θ) - 76.704⎠⋅⎝ ↪
⎜⎢0         ────────────────────────────────────────────────────────────────── ↪
⎜⎢                                                                             ↪
⎜⎢                                                                             ↪
⎜⎢                                                                             ↪
⎜⎢                    ⎛                    ⎛         2            ⎞ ⎛          ↪
⎜⎢   1.19306717673448⋅⎝0.00440553821187624⋅⎝12.96⋅cos (θ) - 76.704⎠⋅⎝τ⋅(11.076 ↪
⎜⎢0  ───────────────────────